Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [2]:
#@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY = True  #@param {type:"boolean"}
WORKSPACE = 'Comfy'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY'] = UPDATE_COMFY

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/Comfy"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup Comfy =- && git clone https://github.com/zhuoyueai/Comfy
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY']:
  !echo -= Updating Comfy =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers==0.0.20 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

-= Initial setup Comfy =-
Cloning into 'Comfy'...
remote: Enumerating objects: 10792, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 10792 (delta 51), reused 52 (delta 32), pack-reused 10683
Receiving objects: 100% (10792/10792), 4.33 MiB | 19.01 MiB/s, done.
Resolving deltas: 100% (7265/7265), done.
/content/Comfy
-= Updating Comfy =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB

Download some models/checkpoints/vae or custom comfy nodes (uncomment the commands for the ones you want)

In [3]:
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P ./models/checkpoints/
!wget -c https://huggingface.co/zhuoyueai/zhuoyueaixl/resolve/main/zhuoyueaiXL_lora_upscale-000008.safetensors -P ./models/loras/
!wget -c https://huggingface.co/zhuoyueai/zhuoyueaixl/resolve/main/yuzhaiaiXL_lora_upscale-000008.safetensors -P ./models/loras/

--2024-05-01 06:08:26--  https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors
Resolving huggingface.co (huggingface.co)... 18.239.50.80, 18.239.50.49, 18.239.50.103, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.80|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/7f/2f/7f2fe2e27137549cd28e570e5bac269b49ebcf1e0e47279c7a941ebe5c948e02/31e35c80fc4829d14f90153f4c74cd59c90b779f6afe05a74cd6120b893f7e5b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sd_xl_base_1.0.safetensors%3B+filename%3D%22sd_xl_base_1.0.safetensors%22%3B&Expires=1714802906&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDgwMjkwNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83Zi8yZi83ZjJmZTJlMjcxMzc1NDljZDI4ZTU3MGU1YmFjMjY5YjQ5ZWJjZjFlMGU0NzI3OWM3YTk0MWViZTVjOTQ4ZTAyLzMxZTM1YzgwZmM0ODI5ZDE0ZjkwMTUzZjRjNzRjZDU

### Run Comfy with cloudflared (Recommended Way)




In [4]:
# !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
# !dpkg -i cloudflared-linux-amd64.deb

# import subprocess
# import threading
# import time
# import socket
# import urllib.request

# def iframe_thread(port):
#   while True:
#       time.sleep(0.5)
#       sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#       result = sock.connect_ex(('127.0.0.1', port))
#       if result == 0:
#         break
#       sock.close()
#   print("\nComfy finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

#   p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#   for line in p.stderr:
#     l = line.decode()
#     if "trycloudflare.com " in l:
#       print("This is the URL to access Comfy:", l[l.find("http"):], end='')
#     #print(l, end='')


# threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

# !python main.py --dont-print-server

### Run Comfy with localtunnel

> Indented block






In [ ]:
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfy finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.952s
Total VRAM 15102 MB, total RAM 12979 MB
xformers version: 0.0.20
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
VAE dtype: torch.float32
Using xformers cross attention
****** User settings have been changed to be stored on the server instead of browser storage. ******
****** For multi-user setups add the --multi-user CLI argument to enable multiple user profiles. ******

Import times for custom nodes:
   0.0 seconds: /content/Comfy/custom_nodes/websocket_image_save.py


Comfy finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)

The password/enpoint ip for localtunnel is: 35.204.231.8
your url is: https://salty-results-study.loca.lt
got prompt
model_type EPS
Using xformers attention in VAE
Using xformers attention in VAE
clip missing: ['clip_l.logit_scale', 'clip_l.transform

### Save images to output.zip file

> Indented block






In [ ]:
!zip -r /content/output.zip /content/Comfy/output

### Run ComfyUI with colab iframe (use only in case the previous way with localtunnel doesn't work)

You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.

If you want to open it in another window use the link.

Note that some UI features like live image previews won't work because the colab iframe blocks websockets.

In [ ]:
# import threading
# import time
# import socket
# def iframe_thread(port):
#   while True:
#       time.sleep(0.5)
#       sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#       result = sock.connect_ex(('127.0.0.1', port))
#       if result == 0:
#         break
#       sock.close()
#   from google.colab import output
#   output.serve_kernel_port_as_iframe(port, height=1024)
#   print("to open it in a window you can open this link here:")
#   output.serve_kernel_port_as_window(port)

# threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

# !python main.py --dont-print-server